In [1]:
####This notebook required run on  parallel algorithms which base on MPI####
import  numpy  as  np
import libpysal as ps
from stwr.gwr import GWR, MGWR,STWR
from stwr.sel_bw import *
from stwr.utils import shift_colormap, truncate_colormap
import geopandas as gp
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import pyplot
import pandas as pd
import math
from matplotlib.gridspec import GridSpec
import time
import csv 
import copy 

In [2]:
import rasterio
import rasterio.plot
import rasterio.features
import rasterio.warp
import pyproj

In [3]:
#读入数据需要有这些
cal_coords_list =[]
cal_y_list =[]
cal_X_list =[]
delt_stwr_intervel =[0.0]
csvFile = open("./Data_STWR/RealWorldData/precip_isotope_D2.csv", "r")
df = pd.read_csv(csvFile,header = 0,names=['Longitude','Latitude','Elevation','ppt','tmean','d2h','timestamp'],
                 dtype = {"Longitude" : "float64","Latitude":"float64",
                          "Elevation":"float64","ppt":"float64","tmean":"float64","d2h":"float64",
                          "timestamp":"float64"},
                 skip_blank_lines = True,
                 keep_default_na = False)
df.info()
df = df.sort_values(by=['timestamp'])  
all_data = df.values
tick_time = all_data[0,-1]
cal_coord_tick = []
cal_X_tick =[]
cal_y_tick =[]
time_tol = 1.0e-7

lensdata = len(all_data)
for row in range(lensdata):
    cur_time = all_data[row,-1]
    if(abs(cur_time-tick_time)>time_tol):
        cal_coords_list.append(np.asarray(cal_coord_tick))
        cal_X_list.append(np.asarray(cal_X_tick))
        cal_y_list.append(np.asarray(cal_y_tick))
        delt_t = cur_time - tick_time
        delt_stwr_intervel.append(delt_t) 
        tick_time =cur_time
        cal_coord_tick = []
        cal_X_tick =[]
        cal_y_tick =[]
    coords_tick = np.array([all_data[row,0],all_data[row,1]])
    cal_coord_tick.append(coords_tick)

    x_tick = np.array([all_data[row,2],all_data[row,3],all_data[row,4]])
    cal_X_tick.append(x_tick)
    y_tick = np.array([all_data[row,5]])
    cal_y_tick.append(y_tick)
#最后在放一次
#gwr解出最后一期 
cal_cord_gwr = np.asarray(cal_coord_tick)
cal_X_gwr  = np.asarray(cal_X_tick)
cal_y_gwr = np.asarray(cal_y_tick)  
cal_coords_list.append(np.asarray(cal_coord_tick))
cal_X_list.append(np.asarray(cal_X_tick))
cal_y_list.append(np.asarray(cal_y_tick))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 7 columns):
Longitude    199 non-null float64
Latitude     199 non-null float64
Elevation    199 non-null float64
ppt          199 non-null float64
tmean        199 non-null float64
d2h          199 non-null float64
timestamp    199 non-null float64
dtypes: float64(7)
memory usage: 11.0 KB


In [4]:
#stwr 
stwr_selector_ = Sel_Spt_BW(cal_coords_list, cal_y_list, cal_X_list,#gwr_bw0,
                            delt_stwr_intervel,spherical = True)
#(1)Here we use parallel algorithms.
#optalpha,optsita,opt_btticks,opt_gwr_bw0 = stwr_selector_.search(nproc = 12) 
#stwr_model = STWR(cal_coords_list,cal_y_list,cal_X_list,delt_stwr_intervel,
                  #optsita,opt_gwr_bw0,tick_nums=opt_btticks,alpha =optalpha,spherical = True,recorded=1)
#(2)We use oridinary algorithms
optalpha,optsita,opt_btticks,opt_gwr_bw0 = stwr_selector_.search() 
stwr_model = STWR(cal_coords_list,cal_y_list,cal_X_list,delt_stwr_intervel,optsita,opt_gwr_bw0,tick_nums=opt_btticks+1,alpha =optalpha,spherical = True,recorded=1)

In [5]:
stwr_results = stwr_model.fit()
print(stwr_results.summary())
stwr_scale = stwr_results.scale 
stwr_residuals = stwr_results.resid_response

Model type                                                         Gaussian
Number of observations:                                                 199
Number of covariates:                                                     4

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                          58711.528
Log-likelihood:                                                    -497.417
AIC:                                                               1002.834
AICc:                                                              1005.417
BIC:                                                              58218.937
R2:                                                                   0.595
Adj. R2:                                                              0.584

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

In [6]:
#gwr  数据只有最后一期
gwr_selector = Sel_BW(cal_cord_gwr, cal_y_gwr, cal_X_gwr,spherical = True)
gwr_bw= gwr_selector.search(bw_min=2)
gwr_model = GWR(cal_cord_gwr, cal_y_gwr, cal_X_gwr, gwr_bw,spherical = True)
gwr_results = gwr_model.fit()
print(gwr_results.summary())
gw_rscale = gwr_results.scale 
gwr_residuals = gwr_results.resid_response

Model type                                                         Gaussian
Number of observations:                                                 109
Number of covariates:                                                     4

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                          58711.528
Log-likelihood:                                                    -497.417
AIC:                                                               1002.834
AICc:                                                              1005.417
BIC:                                                              58218.937
R2:                                                                   0.595
Adj. R2:                                                              0.584

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------